In [15]:
import flamb
from flamb import Variable
from flamb import nn
from flamb import functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.setrecursionlimit(500000)

In [12]:
def f(x):
    return x**2

x = Variable(5, requires_grad=True)
learning_rate = 1e-2

for _ in range(500):        
    loss = f(x)
    loss.backward()
    with flamb.no_grad():
        x -= learning_rate*x.grad
    x.requires_grad = True

print(f"The minimum of the function is {round(f(x).value, 3)} and is obtained with {round(x.value, 3)}")

The minimum of the function is 0.0 and is obtained with 0.0


In [13]:
x = Variable(5, requires_grad=True)
learning_rate = 1e-2
params = flamb.to_tensor([x])
optimizer = flamb.nn.optimizers.SGD(params, learning_rate=learning_rate)

for _ in range(500):        
    loss = f(x)
    loss.backward()
    optimizer.step()

print(f"The minimum of the function is {round(f(x).value, 3)} and is obtained with x={round(x.value, 3)}")

The minimum of the function is 0.0 and is obtained with x=0.0


In [14]:
def g(x, y):
    return x**2 + (y - 3)**2

x = Variable(5, requires_grad=True)
y = Variable(2, requires_grad=True)
learning_rate = 1e-2
params = flamb.to_tensor([x, y])
optimizer = flamb.nn.optimizers.SGD(params, learning_rate=learning_rate)

for _ in range(500):        
    loss = g(x, y)
    loss.backward()
    optimizer.step()


print(f"The minimum of the function is {round(g(x, y).value, 3)} and is obtained with x={round(x.value, 3)} and y={round(y.value, 3)}")

The minimum of the function is 0.0 and is obtained with x=0.0 and y=3.0


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 20)
        self.layer2 = nn.Linear(20, 30)
        self.layer3 = nn.Linear(30, 1)
        self.initialize_parameters()

    def __call__(self, x):
        x = F.ReLU(self.layer1(x))
        x = F.ReLU(self.layer2(x))
        x = self.layer3(x)
        return x

model = Model()
loss_object = nn.losses.MSE()
optimizer = nn.optimizers.SGD(model.parameters, learning_rate=1e-4)

EPOCHS = 20

losses = []
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    total_loss = 0
    for _ in tqdm(range(5)):
        x = flamb.ones((8, 10))
        output = model(x)
        target = flamb.zeros((8,1))
        loss = loss_object(output, target)
        loss.backward()
        optimizer.step()
        
        total_loss += loss

    losses.append(total_loss.get_value())
    print(f"Loss : {total_loss}")

plt.plot([i for i in range(1, EPOCHS+1)], losses)
plt.title("Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()
